In [1]:
import pandas as pd 
import io 
import json
pd.options.display.max_rows = 200

# helpers

In [2]:
def _process_data_(file):
    with open(file, 'r') as file:
        log_content = file.read()
    sections = log_content.split('Sandbox logs:')[1].split('Activities log:')
    sandbox_log =  sections[0].strip()
    activities_log = sections[1].split('Trade History:')[0]
    # sandbox_log_list = [json.loads(line) for line in sandbox_log.split('\n')]
    trade_history =  json.loads(sections[1].split('Trade History:')[1])
    # sandbox_log_df = pd.DataFrame(sandbox_log_list)
    market_data_df = pd.read_csv(io.StringIO(activities_log), sep=";", header=0)
    trade_history_df = pd.json_normalize(trade_history)
    return market_data_df, trade_history_df

In [3]:
def get_prev_returns(df, col, its):
    prev_col = f"{col}_prev_{its}_its"
    df[prev_col] = df[col].shift(its)
    df[f"{col}_returns_from_{its}_its_ago"] = (df[col] - df[prev_col]) / df[prev_col]
    df.drop(columns=[prev_col], inplace=True)
    return df

def get_future_returns(df, col, its):
    future_col = f"{col}_future_{its}_its"
    df[future_col] = df[col].shift(-its)
    df[f"{col}_returns_in_{its}_its"] = (df[future_col] - df[col]) / df[col]
    df.drop(columns=[future_col], inplace=True)
    return df

def get_centered_returns(df, col, its):
    future_col = f"{col}_future_{its}_its"
    df[future_col] = df[col].shift(-its)
    prev_col = f"{col}_prev_{its}_its"
    df[prev_col] = df[col].shift(its)
    df[f"{col}_returns_centered_with_{its}_its"] = (df[future_col] - df[prev_col])/df[prev_col]
    df.drop(columns=[prev_col], inplace=True)
    df.drop(columns=[future_col], inplace=True)
    return df

# process 23 data

In [85]:
day = 3

In [86]:
df = pd.read_csv(f"2023_data_logs/r{day}.csv", sep=';')

In [87]:
df_diving = df[df['product'] == 'DIVING_GEAR']

In [88]:
df_diving.columns

Index(['day', 'timestamp', 'product', 'bid_price_1', 'bid_volume_1',
       'bid_price_2', 'bid_volume_2', 'bid_price_3', 'bid_volume_3',
       'ask_price_1', 'ask_volume_1', 'ask_price_2', 'ask_volume_2',
       'ask_price_3', 'ask_volume_3', 'mid_price', 'profit_and_loss'],
      dtype='object')

In [89]:
df_pred = df_diving[['day','timestamp', 'mid_price']].copy()

In [90]:
df_pred = get_future_returns(df_pred, 'mid_price', 1).reset_index(drop=True)
df_pred = get_future_returns(df_pred, 'mid_price', 2).reset_index(drop=True)

In [91]:
df_pred['ROSES_pred_returns_in_1_its'] = df_pred['mid_price_returns_in_1_its']* 3.07

In [92]:
df_pred['ROSES_pred_returns_in_1_its'].abs().describe()

count    9999.000000
mean        0.000152
std         0.000115
min         0.000000
25%         0.000061
50%         0.000123
75%         0.000215
max         0.000844
Name: ROSES_pred_returns_in_1_its, dtype: float64

In [93]:
df_pred.columns

Index(['day', 'timestamp', 'mid_price', 'mid_price_returns_in_1_its',
       'mid_price_returns_in_2_its', 'ROSES_pred_returns_in_1_its'],
      dtype='object')

In [94]:
df_pred[['ROSES_pred_returns_in_1_its']].to_csv('roses_pred_returns_r5.csv', index=False)

In [95]:
df_pred

,day,timestamp,mid_price,mid_price_returns_in_1_its,mid_price_returns_in_2_its,ROSES_pred_returns_in_1_its
0,3,0,100188.5,0.000000,0.000065,0.000000
1,3,100,100188.5,0.000065,0.000170,0.000199
2,3,200,100195.0,0.000105,0.000175,0.000322
3,3,300,100205.5,0.000070,0.000065,0.000214
4,3,400,100212.5,-0.000005,-0.000025,-0.000015
...,...,...,...,...,...,...
9995,3,999500,100276.0,0.000050,0.000135,0.000153
9996,3,999600,100281.0,0.000085,0.000160,0.000260
9997,3,999700,100289.5,0.000075,0.000070,0.000230
9998,3,999800,100297.0,-0.000005,NaN,-0.000015


In [96]:
pred_returns_list = df_pred['ROSES_pred_returns_in_1_its'].tolist()

In [97]:
import numpy as np
roses_pred_price = [14411.5]
for i in pred_returns_list:
    roses_pred_price.append(roses_pred_price[-1] * (1 + i))
rose_pred_returns_price = np.diff(roses_pred_price)
df_pred['ROSES_pred_returns_in_1_its_price'] = np.round( 2 * rose_pred_returns_price)/2

In [98]:
pred_returns_pct_list = df_pred['ROSES_pred_returns_in_1_its'].tolist()
pred_returns_price_list = df_pred['ROSES_pred_returns_in_1_its_price'].tolist()

In [99]:
pred_returns_price_agg = []
sign = 0
sum_return = 0
for i in range(len(pred_returns_price_list)-1, -1 ,-1):
    if np.sign(pred_returns_price_list[i]) != sign:
        sum_return = 0
    sum_return += pred_returns_price_list[i]
    pred_returns_price_agg.append(sum_return)
    sign = np.sign(pred_returns_price_list[i])
pred_returns_price_agg = pred_returns_price_agg[::-1]
df_pred['pred_returns_price_agg'] = pred_returns_price_agg

In [100]:
pred_returns_pct_agg = []
sign = 0
prod_return = 1
for i in range(len(pred_returns_pct_list)-1, -1 ,-1):
    if np.sign(pred_returns_pct_list[i]) != sign:
        prod_return = 1
    prod_return *= 1 + pred_returns_pct_list[i]
    pred_returns_pct_agg.append(prod_return - 1)
    sign = np.sign(pred_returns_pct_list[i])
pred_returns_pct_agg = pred_returns_pct_agg[::-1]
df_pred['pred_returns_pct_agg'] = pred_returns_pct_agg

In [101]:
df_pred.head(30)

,day,timestamp,mid_price,mid_price_returns_in_1_its,mid_price_returns_in_2_its,ROSES_pred_returns_in_1_its,ROSES_pred_returns_in_1_its_price,pred_returns_price_agg,pred_returns_pct_agg
0,3,0,100188.5,0.000000,0.000065,0.000000,0.0,0.0,0.000000
1,3,100,100188.5,0.000065,0.000170,0.000199,3.0,10.5,0.000736
2,3,200,100195.0,0.000105,0.000175,0.000322,4.5,7.5,0.000536
3,3,300,100205.5,0.000070,0.000065,0.000214,3.0,3.0,0.000214
4,3,400,100212.5,-0.000005,-0.000025,-0.000015,-0.0,0.0,-0.000168
5,3,500,100212.0,-0.000020,-0.000050,-0.000061,-1.0,-2.5,-0.000153
6,3,600,100210.0,-0.000030,0.000035,-0.000092,-1.5,-1.5,-0.000092
7,3,700,100207.0,0.000065,0.000100,0.000199,3.0,7.5,0.000521
8,3,800,100213.5,0.000035,0.000105,0.000107,1.5,4.5,0.000322
9,3,900,100217.0,0.000070,0.000055,0.000214,3.0,3.0,0.000214


In [102]:
trades_list = []
for ret in pred_returns_price_agg:
    if ret > 0.5 and ret <= 1.5:
        trades_list.append('b')
    elif ret > 1.5:
        trades_list.append("B")
    elif ret < -0.5 and ret >= -1.5: 
        trades_list.append('s')
    elif ret < -1.5:
        trades_list.append("S")
    else:
        trades_list.append('h')

In [103]:
trades_string = ''.join(trades_list)

In [104]:
trades_string

'hBBBhSsBBBshhhhBbSsBbhhBSBBBBSsBBBShBBBBBhbShBBSSSSShBBhSbhBBShBBhSBbhBBBhhbSSSbSSsBSShhBSSBBBBBsbhSSSSSBhSShSSShBSshSBhShSBhBSBBSSshsBBSSSSSSShhSSSbsBBbhhShBSshBbSShhBhhSSSBBSSshSsBhSSsBSBhBSSShhshbSBSShshSSSSbSSShSBbsBBSSBSBBSBshBSshSshhhSSSSBBhSsBBBBShhSShBBhhShBsBBSSshBshBSSsshhSBsbhBBSsbhBSSBSBSBBBsBBhBBBhbSSbShSSSbShbshBBBBBbSbSShBBBBBSShBhSSSSSBhShBBSSBSShBBShShSBBshBSSshBSSSSSSSbhshhSSBSSshSSSSSSBShBSSSSSSsBBBBBhSSBBBhBhShhbSSBsBBBBhBBBShBBBBShBbSBbSsBBSSshhShSShhhbhsBhBSBSSSSSSSsbSBSBhSBhSBsBBSSbSBSBBSSBShBBhSBBBSBbsbhSSBbhSSshhbSSSSShsBSSBBSSBShSSBbhhSsBBSbSSBBSBhBBBhSSSSBBBBSBhSShBhBBBSSBBSBBbsBBBSSshSBSBSShBBSBSSSBSShBShhSSsbSSBBShshBSSSSBBShSSsBBhBsBSSBSbSBShSBSSSSBSShhhsBSSsBBhBBBSBhBBBSSSSBhhShBBbhSShSshBSSBSBBBbSbShhBBhBbSbsBSsBSbsBBSBBsBBShBSSSSBBBbSSBBBBBShBSSsBSshSSbhSSBshSSBSSBshbSSSSBsbSShSBSSSSSBhSSbSBbSsBBBSBSBBsbSsbbhbSSbSSSBBBSSSSBSShsBBbhShSSSBSBSshBSbSSSSShBbsBShsBbhBhSBSsbshBBSsBbSbSBBBSBBbhBShbshBBhSSSSSBshbSsbSBSBBBhhBSSBBbSSSsBBhbSBBSSBBBsbShSBSBBBBShBbS

In [64]:
day = 2
df = pd.read_csv(f"2023_data_logs/r{day}.csv", sep=';')
df_coconut = df[df['product'] == 'COCONUTS']
df_pred_coconut = df_coconut[['day','timestamp', 'mid_price']].copy()
df_pred_coconut = get_future_returns(df_pred_coconut, 'mid_price', 1).reset_index(drop=True)
df_pred_coconut.tail()

,day,timestamp,mid_price,mid_price_returns_in_1_its
9995,2,999500,7910.0,0.000063
9996,2,999600,7910.5,0.000126
9997,2,999700,7911.5,0.000126
9998,2,999800,7912.5,-0.000126
9999,2,999900,7911.5,NaN


In [52]:
df_r4, _ = _process_data_("./2024_data_logs/results_round4.log")
df_r4_coconut = df_r4[df_r4['product'] == 'COCONUT'].copy().reset_index(drop=True)
# Calculate the count of bid and ask prices with the price difference to the mid price
# df_r4_coconuts['mid_price'] = (df_r4_coconuts['bid_price_1'] + df_r4_coconuts['ask_price_1']) / 2

# Initialize dictionaries to store the sum of volumes for each unique price difference
bid_price_diff_volumes = {}
ask_price_diff_volumes = {}

# Iterate through each row to calculate differences and sum volumes for unique differences
for index, row in df_r4_coconut.iterrows():
    bid_prices = [row['bid_price_1'], row['bid_price_2'], row['bid_price_3']]
    ask_prices = [row['ask_price_1'], row['ask_price_2'], row['ask_price_3']]
    bid_volumes = [row['bid_volume_1'], row['bid_volume_2'], row['bid_volume_3']]
    ask_volumes = [row['ask_volume_1'], row['ask_volume_2'], row['ask_volume_3']]
    
    # Process bid prices and volumes
    for i in range(3):
        if pd.notna(bid_prices[i]) and pd.notna(bid_volumes[i]):
            bid_diff = abs(bid_prices[i] - row['mid_price'])
            if bid_diff in bid_price_diff_volumes:
                bid_price_diff_volumes[bid_diff] += bid_volumes[i]
            else:
                bid_price_diff_volumes[bid_diff] = bid_volumes[i]
    
    # Process ask prices and volumes
    for i in range(3):
        if pd.notna(ask_prices[i]) and pd.notna(ask_volumes[i]):
            ask_diff = abs(ask_prices[i] - row['mid_price'])
            if ask_diff in ask_price_diff_volumes:
                ask_price_diff_volumes[ask_diff] += ask_volumes[i]
            else:
                ask_price_diff_volumes[ask_diff] = ask_volumes[i]

# Display the dictionaries containing the sum of volumes for each unique price difference
print("Bid Price Difference Volumes:", bid_price_diff_volumes)
print("Ask Price Difference Volumes:", ask_price_diff_volumes)


FileNotFoundError: [Errno 2] No such file or directory: './2024_data_logs/results_round4.log'

In [53]:
# Calculate the sum and mean of bid and ask volumes for each timestamp
# Group by timestamp to handle the operations for each unique timestamp
df_r4_coconuts_vol = df_r4_coconut.copy()
df_r4_coconuts_vol.fillna(0, inplace=True)

sum_bid_1 = df_r4_coconuts_vol['bid_volume_1'].sum()
sum_bid_2 = df_r4_coconuts_vol['bid_volume_2'].sum()
sum_bid_3 = df_r4_coconuts_vol['bid_volume_3'].sum()
sum_ask_1 = df_r4_coconuts_vol['ask_volume_1'].sum()
sum_ask_2 = df_r4_coconuts_vol['ask_volume_2'].sum()
sum_ask_3 = df_r4_coconuts_vol['ask_volume_3'].sum()


sum_all_bid = sum_bid_1 + sum_bid_2 + sum_bid_3
sum_all_ask = sum_ask_1 + sum_ask_2 + sum_ask_3

print(sum_all_bid, sum_all_ask)
print(f"mean of bid volumes: {sum_all_bid / len(df_r4_coconuts_vol)}")
print(f"mean of ask volumes: {sum_all_ask / len(df_r4_coconuts_vol)}")


1848911.0 1848960.0
mean of bid volumes: 184.8911
mean of ask volumes: 184.896


In [54]:
# Calculate the total weighted spread for bid and ask price difference volumes
if bid_price_diff_volumes:
    total_bid_volume = sum(bid_price_diff_volumes.values())
    weighted_bid_spread = sum(diff * volume for diff, volume in bid_price_diff_volumes.items()) / total_bid_volume if total_bid_volume else 0
else:
    weighted_bid_spread = 0

if ask_price_diff_volumes:
    total_ask_volume = sum(ask_price_diff_volumes.values())
    weighted_ask_spread = sum(diff * volume for diff, volume in ask_price_diff_volumes.items()) / total_ask_volume if total_ask_volume else 0
else:
    weighted_ask_spread = 0

print("Total Weighted Bid Price Spread:", weighted_bid_spread)
print("Total Weighted Ask Price Spread:", weighted_ask_spread)


NameError: name 'bid_price_diff_volumes' is not defined

In [86]:
import numpy as np
coconut_past_price = df_pred_coconut['mid_price'].to_numpy()
spread =[0.99] * len(coconut_past_price)

In [106]:
import math
def optimal_trading_dp(prices, spread, volume_pct):
    n = len(prices)
    price_level_cnt = math.ceil(1/volume_pct)
    left_over_pct = 1 - (price_level_cnt - 1) * volume_pct

    dp = [[float('-inf')] * (price_level_cnt * 2 + 1) for _ in range(n)]  # From -3 to 3, 7 positions
    action = [[''] * (price_level_cnt * 2 + 1) for _ in range(n)]  # To store actions

    # Initialize the starting position (no stock held)
    dp[0][price_level_cnt] = 0  # Start with no position, Cash is 0
    action[0][price_level_cnt] = ''  # No action at start

    def position(j):
        if j > price_level_cnt:
            position = min((j - price_level_cnt) * volume_pct, 1)
        elif j < price_level_cnt:
            position = max((j - price_level_cnt) * volume_pct, -1)
        else:
            position = 0
        return position
    
    def position_list(list):
        return np.array([position(x) for x in list])

    for i in range(1, n):
        for j in range(0, price_level_cnt * 2 + 1):
            # Calculate PnL for holding, buying, or selling
            hold = dp[i-1][j] if dp[i-1][j] != float('-inf') else float('-inf')
            if j == price_level_cnt * 2:
                buy = dp[i-1][j-1] - left_over_pct*prices[i-1] -  left_over_pct*spread if j > 0 else float('-inf')
            elif j == 1:
                buy = dp[i-1][j-1] - left_over_pct*prices[i-1] -  left_over_pct*spread if j > 0 else float('-inf')
            else:
                buy = dp[i-1][j-1] - volume_pct*prices[i-1] - volume_pct*spread if j > 0 else float('-inf')

            if j ==  0:
                sell = dp[i-1][j+1] + left_over_pct*prices[i-1] - left_over_pct*spread if j < price_level_cnt * 2 else float('-inf')
            elif j == price_level_cnt * 2 - 1:
                sell = dp[i-1][j+1] + left_over_pct*prices[i-1] - left_over_pct*spread if j < price_level_cnt * 2 else float('-inf')
            else:
                sell = dp[i-1][j+1] + volume_pct*prices[i-1] - volume_pct*spread if j < price_level_cnt * 2 else float('-inf')
                
            # Choose the action with the highest PnL

            hold_pnl = hold + (j - price_level_cnt) * position(j) * prices[i]
            buy_pnl = buy + (j - price_level_cnt) * position(j) * prices[i]
            sell_pnl = sell + (j - price_level_cnt) * position(j) * prices[i]
            
            # print(hold_pnl, buy_pnl, sell_pnl)
            best_action = max(hold_pnl, buy_pnl, sell_pnl)
            if best_action == hold_pnl:
                dp[i][j] = hold
            elif best_action == buy_pnl:
                dp[i][j] = buy
            else:
                dp[i][j] = sell

            if best_action == hold_pnl:
                action[i][j] = 'h'
            elif best_action == buy_pnl:
                action[i][j] = 'b'
            else:
                action[i][j] = 's'
    # Backtrack to find the sequence of actions
    trades_list = []
    # Start from the position with maximum PnL at time n-1

    pnl = np.array(dp[n-1]) + (position_list(np.arange(0,price_level_cnt*2+1)) * prices[n-1])
    current_position = np.argmax(pnl)
    for i in range(n-1, -1, -1):
        trades_list.append(action[i][current_position])
        if action[i][current_position] == 'b':
            current_position -= 1
        elif action[i][current_position] == 's':
            current_position += 1

    trades_list.reverse()
    trades_list.append('h')
    return dp, trades_list, pnl[np.argmax(pnl)]  # Return the actions and the maximum PnL

# Example usage
dp, trades, max_pnl = optimal_trading_dp(coconut_past_price, 0.99, 185/300)
# print(trades)
print("Max PnL:", max_pnl)

Max PnL: 1822.9471666670502


In [107]:
trades_string = ''.join(trades)
# len(trades_string)
trades_string

'bhhshhhhhhsshhhhhhhhhhhhhhhhhhhbbbhhhhhshbbhhhhshsshhhhhshhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhbhhbbbhhhhhhhhhhhhhhshhhhbhhhhhhhhhhhhhhhhsshshhhhhhbbhhhshhbbhhhhhhhhhhhhhshhhbhhhhhsshhhsshhhbhhhhhhhshhhhbhbhhhhhhhhhshbhhbbhhshhbhshhhhhhhhhbhhhshhhbhshhbhhhhhhhhhhhhsssshhhhbhhhshhhhhhhhhhhbbbbhhhhhhhhhhshhbhhhhhhhshshhhhhhhsshhhhhhhhbbhbbhhhhhhhhhhhhhhhhhhhhhhhhhsssshhhhhbhshhhhhhhhhhhhhhhhhhhhhbhhhhhshhhhbhhhhhbbhhshhhhhbhbhhhsssshhhhbbbbhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhshhshshhhhhbbbhhhhhhhhshhbhhhhhsssshhhhhhhhhhhhhbbhhhsshhhhhhhhhbhbhsshhhbbbhhhhshhbhshsshhhhhhhbshhhhhhhhhhhhhbhhhbhhshhshhhhhhhbhbhhhshbbhhssshhhhhhhhhhhhhhhhhbhhhhhhhbbbhssshhhhshhhbhhhhhhhhshhhhhhhhhbbhshhshhhhhhhhbbbbhhhhhhhhhhhhhhhshshsshhhhhhhhhhhhbbhhhhhhhsshhhhhhhbbbbhhhhhhhhhhhsssshhhhhhhhhbhhhhhhshhhhhhhhhhhhhhhbhbhhhbhhbhhhssshhhshhhhhhhhhhhhhhbhbshhhhhhhhhhbhbbhhhhshhhhhhsshhshhhhhhhhhhbhhhhhhhhhhbbbhhhhhhshhsshshhbbhbhhhbhhhhhhhhhhhhhhhhhhshssshhhhhhhhhhhhhhhhhhhhhhhhbhhbbhsshshhbhbbbhhhhhhhhhhhhhhhhhhhhhhhhhhh

In [99]:
dp[0:3]

[[-inf, -inf, 0, -inf, -inf],
 [-inf, 4885.856166666667, 0, -4887.077166666667, -inf],
 [7923.201666666667,
  4886.164500000001,
  0,
  -4887.077166666667,
  -7925.181666666666]]

In [315]:
df_pred_coconut24 = [9883.5]
for i in df_pred_coconut['mid_price_returns_in_1_its']:
    if pd.isna(i):
        df_pred_coconut24.append(df_pred_coconut24[-1])
    else:
        df_pred_coconut24.append(df_pred_coconut24[-1] * (1 + i))
df_pred_coconut['COCONUT_pred_price'] = np.array(df_pred_coconut24[:-1])
df_pred_coconut['COCONUT_pred_price_diff'] = df_pred_coconut['COCONUT_pred_price'].diff().shift(-1)
df_pred_coconut['COCONUT_actual_price'] = df_r4_coconuts['mid_price']


In [328]:
pred_returns_price_list = df_pred_coconut['COCONUTS_price_diff'].to_numpy()
pred_returns_price_agg = []
sign = 0
sum_return = 0
for i in range(len(pred_returns_price_list)-1, -1 ,-1):
    if np.sign(pred_returns_price_list[i]) != sign:
        sum_return = 0
    sum_return += pred_returns_price_list[i]
    pred_returns_price_agg.append(sum_return)
    sign = np.sign(pred_returns_price_list[i])
pred_returns_price_agg = pred_returns_price_agg[::-1]
df_pred_coconut['COCONUT_pred_price_agg'] = pred_returns_price_agg

In [332]:
trades_list = []
for ret in pred_returns_price_agg:
    if ret > 0.5 and ret <= 2:
        trades_list.append('b')
    elif ret > 2:
        trades_list.append("B")
    elif ret < -0.5 and ret >= -2: 
        trades_list.append('s')
    elif ret < -2:
        trades_list.append("S")
    else:
        trades_list.append('h')

In [333]:
trades_string = ''.join(trades_list)
trades_string

'BbbsshhhbhhSssbshsbshhbhhhSshhshhhbshbbshhhBBbbsbhshsbhbshhbshSshssbSshhhhbsbsbhshhhhbhSsshhsbhbhhhhhbhhBhsshbhhhshBbhBBbsBBBbhhhsbbhhbShhhhsshBBBbSshBBBbhhbhhhhshBbhhsshhhhbbhsbbhsshssbhhhbshbsbsshhhbhhhhhhhhbssbshhBBbhSsBbshbshbshhsbhhbsshsBhshhbbbshshbhhbhbhhhSbhsshhbhsbshhshshsshhsBBbhbhhhbbhbsshhbhhhBbbhhsshbshBbSShhssbbshhhhBbbhhhhhhbhhhhhBbshhshhbhBBBSSShbSssbbSshhhhbhhbhhshhshhbshhbhhbbhSshshhhhhhsBBbbshhhhhbsbhBbhhhSSSSShhBBBBbbhhsbhbhhBbhsBbhbhshhhbhbbhhbshbsbShhhhSBbbhhssBbsbshsbbsBBbbshshSshhhhbhhhhshBBbbshhSshshhhshhbbhSshshhbbhhbSshBbsbsshhhshhSbSSSSshsshhhhbshhshBbhsbbsbhSshhsbsbhhbsshBBbbSSSsbhhhhssbSSsshSsbshhhhssBBbbhhSsbhbshhshbhhhsbbhhhhhshhSSsBbbshbhhshbhshsshBBbhhhbshbhhhsBbhsbsbhSsshhhbsshhbshbhhhhbhhSsbsbhhhShhbhhbhhhhhhBhbhshhSSshhhhhsbbhhhhhSshbshhhshhhbshhhhhhhhbbshBhbhSshbbSshshhshsbhshhsbsBhhshhhhbhhshsbhhbhbsbhhhhhhshhhhsshsbSSshhhbhshhbhSssBBbhBbhBhhbbshshhhhhbbssbbhBBhhhhhhhshhhbhbhbhhshhhhhhhhhshhbshhhbhsbshsbsshBbhsbSsshshBbbbshhhbhbhhhhhbhshbshhbhshh